## Running Stan Models
**Note:** This is a tutorial notebook that walks through the second half of the `mcmcjob.py` script. Although this notebook can be used to run Stan models on a personal machine, it cannot be used on the HBS cluster - juypter notebooks are not supported on the cluster at this time. Therefore, please use the `mcmcjob.py` script when submitting jobs on the cluster. This notebook is intended as a reference.

### Step 1: Import Libraries, Data, and Custom Functions
The second half of the `mcmcjob.py` script requires the following libraries:
* pickle
* sys
* pystan
* arviz

Pickle will be used to load in our pickled data that we prepared for Stan in the previous tutorial notebook. Pystan will be used as an API wrapper around the Stan language, which we will use to write our models. **Please use Pystan 2.19.1.1**. You can use a newer release, but note that Pystan 3 is a rewrite and many of the commands I use in this tutorial are not backwards-compatible. There are plenty of good reasons to upgrade to 3.0+, but I've preferred to stick with 2.0 because 3.0 removed many useful features, such as variational inference. Let's import the libraries:

In [1]:
import pickle
import sys
import pystan
import arviz as az

INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


Next, we will import the data that we previously pickled.

In [2]:
with open('modeldata.pkl', 'rb') as f:
    data = pickle.load(f)

Last, we will import some custom python functions. 

`inits.py` contains a function `init_f()` that define all of the possible initial values that we will have Stan begin sampling at. Stan uses a Markov Chain Monte Carlo (MCMC) algorithm to identify the posterior distribution of our modeling parameters given the data, and MCMC traverses the gradient of the log target density by proposing new, hopefully more likely parameter values. The algorithm has to start somewhere, so we will feed in some probable starting values. 

`facesddm_stan.py` contains our Stan modeling code. I will write a separate tutorial on the Stan langauge. For now, know that we need to import specific Stan models from this script to fit those models to our data. We are fitting the `ddmraceallcode` model in this tutorial, which allows all of our modeling parameters to vary freely with valence and race ratio (with the exception of $\tau$, non-decision time).

In [21]:
from inits import init_f
if "facesddm_stan" in sys.modules:
    sys.modules.pop('facesddm_stan')
from facesddm_stan import ddmcode

### Step 2: Compile the Stan Model
The next step, while straightfoward code-wise, can be quite difficult to get working on the backend. First, make sure that you've properly installed pystan with all of its dependencies. This includes cython, a superset of python that supports the C language. Second, you need to install a C++ compiler, such as GCC or clang-cl. Please see documentation here for installation instructions specific to your operating system: https://pystan2.readthedocs.io/en/latest/installation_beginner.html

Once you've installed pystan, its dependencies, and a C++ compiler, you are ready to compile your Stan model. This will take a minute or two to complete.

In [23]:
ddm_sm = pystan.StanModel(model_code=ddmcode, model_name='DDM')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL DDM_8efbd84064def26ae6563d88a5c70a4e NOW.


### Step 3: Fit the Model to Data with MCMC
As before, this next step is simple to execute but arguably the bulk of your workload. This next line will begin MCMC sampling using your model code and data. From here on, I will *NOT* be running the rest of the notebook, as it would take a few days to complete on a personal machine and depending on your RAM and number of cores, could brick your machine for the duration. It's much faster on a computing cluster and, by virtue of being on a cluster, doesn't tax your personal machine.

In [ ]:
ddm_fit = ddm_sm.sampling(data=data, init=init_f('ddmraceallcode', data['N']), iter=20000, 
                          warmup=10000, chains=4, n_jobs=4, seed=101, refresh=1)

In [ ]:
dd_fit = ddm_sm.vb(data=data, init=init_f('ddmcode', data['N']), 
                   iter=2000)

Let's take a moment to understand the attributes of the sampling function. `data` is straightforward. `init` is the starting values of each model parameter, as previously described. 

`iter` is the number of iterations or samples that you'd like to run. MCMC can take time to converge onto the posterior distribution of most likely parameter values, so we need to run a lot of iterations to ensure that we reach that target of maximal likelihood and still have iterations to spare for sampling the target. How many iterations is a good number? Unfortunately, there's no hard and fast rule for this - the number of iterations that are reported in the literature (when they're reported) varies wildly between models from as low as $2000$ upwards to $100000$. At the end of the day, what we want is a good number of samples in the posterior, or the *effective sample size*. A general suggestion is that your effective sample size is at least 0.1% of your total samples. Sometimes, increasing the number of iterations can improve your effective sample size. Othertimes, you might want to reconsider how you've written your model. I like to keep the number of iterations run on the high end since it's more likely to give me the effective sample size I need and looks better for publication.

`warmup` is the number of iterations in `iter` that we want to discard from the beginning. This has historically been referred to as "burn-in". You might ask, why do we want to discard samples that we waited so long to acquire? The reason is that a lot of those samples that we get in the beginning of MCMC are not in the target posterior. We do *NOT* want to use these samples for inference. A standard rule of thumb is to discard half of your iterations as warmup.

`chains` is the number of MCMC chains that you'd like to run. If we have $4$ chains and $20000$ iterations, that means we're running $20000$ iterations $4$ times. Why do this? Sometimes, there are multiple local maximal likelhoods in the joint posterior - we do *NOT* want to converge on different joint posteriors as that would make interpretation very difficult. To check for this potential convergence issue, we need to run MCMC multiple times and ensure that we land around the same values each time. This means running multiple chains. $4$ chains is fairly standard.

`n_jobs` is the number of cores you'd like to use for running your chains in parallel. To run your chains most efficiently, dedicate one core per chain. Unfortunately, we cannot parallelize beyond this without threading or using a message passing interface for breaking up the likelihood calculations. I am looking into setting up a MPI on the cluster, which will make better use of the available compute and save us a ton of time running models.

`seed` sets a seed for reproducing our MCMC samples. `refresh` prints iteration progress to your stdout.

**Note:** If you were running this on the HBSGrid, you could use the `bsub` command to submit `mcmcjob.py` to a batch scheduling queue: `bsub -q long -R "rusage[mem=64000]" -M 64000 mcmcjob.py`. Modify the command with the queue you like to join and the amount of memory requested.

### Step 4: Saving your Compiled Code and Model Fit
After Stan has finally finished sampling, I like to save the code and fit to both a pickle and a netcdf. Pickle is nice because it preserves the stanfit object, but can't be shared between operating systems. NetCDF needs to convert the stanfit to an InferenceData object but can be shared universally. We will use the arviz package to work with NetCDF.

In [ ]:
with open("simpleraceallddm.pkl", "wb") as f:
    pickle.dump({'model' : ddm_sm, 'fit' : ddm_fit}, f, protocol=-1)
azdf = az.from_pystan(posterior=ddm_fit, 
                      observed_data=['rt'], 
                      log_likelihood={'logy': 'log_lik'})
azdf.to_netcdf('simpleraceallddm.nc')